# Your first scraper
In this project, we will guide you step by step through the process of:

1. creating a self-contained development environment.
1. retrieving some information from an API (a website for computers)
2. leveraging it to scrape a website that does not provide an API
3. saving the output for later processing

Here we query an API for a list of countries and their past leaders. We then extract and sanitize their short bio from Wikipedia. Finally, we save the data to disk.

This task is often the first (coding) step of a datascience project and you will often come back to it in the future.

You will study topics such as *scraping*, *data structures*, *regular expressions*, *concurrency* and *file handling*. We will point out useful resources at the appropriate time. 

Let's dive in!

## 0. Creating a clean environment

Use the [`venv`](https://docs.python.org/3/library/venv.html) command to create a new environment called `wikipedia_scraper_env`.

Activate it and add it to you `.gitignore` file. 

You will find more info about virtual environments in the course content and on the web.

## 1. API Scraping

### 1a. A simple API query
You will start with the basics: how to do a simple request to an [API endpoint](../../2.python/2.python_advanced/05.Scraping/5.apis.ipynb).

You will use the [requests](https://requests.readthedocs.io/en/latest/) external library through the `import` keyword. NOTE: external libraries need to be installed first. Check the [request Quickstart](https://requests.readthedocs.io/en/latest/user/quickstart/) section of the documentation to:

1. Use the `get()` method to connect to this endpoint: https://country-leaders.onrender.com/status
2. Check if the `status_code` is equal to 200, which means OK.
    * if OK, `print()` the `text`` of the response.
    * if not, `print()` the `status_code`. 

Here is an explanation of [HTTP status codes](https://en.wikipedia.org/wiki/List_of_HTTP_status_codes).


In [38]:
# import the requests library (1 line)
import requests

# assign the root url (without /status) to the root_url variable for ease of reference (1 line)
root_url = "https://country-leaders.onrender.com"

# assign the /status endpoint to another variable called status_url (1 line)
endpoint = "status"

# query the /status endpoint using the get() method and store it in the req variable (1 line)
response = requests.get(f"{root_url}/{endpoint}")

# check the status_code using a condition and print appropriate messages (4 lines)
if response.status_code == 200 :
    print(response.text)
else: 
    print(response.status_code)
    

"Alive"


### 1b. Dealing with JSON

[JSON](https://quickref.me/json) is the preferred format to deal with data over the web. You cannot avoid it so you would better get acquainted.

Connect to another endpoint called `/countries` but this time the API will return data in the JSON format. 


In [37]:
# Set the countries_url variable (1 line)
import requests
root_url = "https://country-leaders.onrender.com"

endpoint = "countries"

# query the /countries endpoint using the get() method and store it in the req variable (1 line)
response = requests.get(f"{root_url}/{endpoint}")

# Get the JSON content and store it in the countries variable (1 line)
data = response.json()
# display the request's status code and the countries variable (1 line)
print(response.status_code)
print(data)
print(response.cookies)


403
{'message': 'The cookie is missing'}
<RequestsCookieJar[]>


### 1c. Cookies anyone?

It looks like the access to this API is restricted...
Query the `/cookie` endpoint and extract the appropriate field to access your cookie.

You will need to use this cookie in each of the following API requests.

In [11]:
login_response = requests.get("https://country-leaders.onrender.com/cookie")  # cookie istegi gonderdim
print(login_response.cookies) # istek burda saklanir  cookie jar sunucudan gelen cookie
session_cookies = login_response.cookies #sunucudan gelen cookie nesnesini degiskende sakla
countries_response = requests.get("https://country-leaders.onrender.com/countries", cookies = session_cookies) # bu cookie degsikeni kullanarak ulkeleri iste
print(countries_response.status_code)
print(countries_response.json())

<RequestsCookieJar[<Cookie user_cookie=1d6f4bb8-69b3-4191-8fbe-66a67396c187 for country-leaders.onrender.com/>]>
200
['be', 'us', 'ru', 'fr', 'ma']


In [39]:
# Set the cookie_url variable (1 line)
cookie_url = f"{root_url}/cookie"

# Query the enpoint, set the cookies variable and display it (2 lines)
response = requests.get(cookie_url) 
cookies = response.cookies
print(cookies)

<RequestsCookieJar[<Cookie user_cookie=3de8a05e-de73-4e2b-bb6e-669623b805d8 for country-leaders.onrender.com/>]>


Try to query the countries endpoint using the cookie, save the output and print it.

In [40]:
# query the /countries endpoint, assign the output to the countries variable (1 line)
# display the countries variable (1 line)

countries_url = f"{root_url}/countries"
countries_response = requests.get(countries_url,cookies = cookies)
countries = countries_response.json()
print(countries)


['be', 'us', 'ru', 'fr', 'ma']


Chances are the cookie has expired... Thanksfully, you got a nice error message. For now, simply execute the last 2 cells quickly so you get a result.

### 1d. Getting the actual data from the API

Query the `/leaders` endpoint.

In [41]:
# Set the leaders_url variable (1 line)
leaders_url = f"{root_url}/leaders"

# query the /leaders endpoint, assign the output to the leaders variable (1 line)
leader_response = requests.get(leaders_url,cookies=cookies)
leaders = leader_response.json()

# display the leaders variable (1 line)
print(leaders)

{'message': 'Please specify a country'}


It looks like this endpoint requires additional information in order to return its result. Check the API [*documentation*](https://country-leaders.onrender.com/docs) in your web browser.

Change the query to accept *parameters*. You should know where to find help by now.

In [42]:

params = {"country" : "us"}

#query the /leaders endpoint using cookies and parameters (take any country in countries)
leader_response = requests.get(leaders_url,cookies=cookies,params=params)

# assign the output to the leaders variable (1 line)
leaders = leader_response.json()

# display the leaders variable (1 line)
print(leaders)

[{'id': 'Q23', 'first_name': 'George', 'last_name': 'Washington', 'birth_date': '1732-02-22', 'death_date': '1799-12-14', 'place_of_birth': 'Westmoreland County', 'wikipedia_url': 'https://en.wikipedia.org/wiki/George_Washington', 'start_mandate': '1789-04-30', 'end_mandate': '1797-03-04'}, {'id': 'Q76', 'first_name': 'Barack', 'last_name': 'Obama', 'birth_date': '1961-08-04', 'death_date': None, 'place_of_birth': 'Kapiolani Medical Center for Women and Children', 'wikipedia_url': 'https://en.wikipedia.org/wiki/Barack_Obama', 'start_mandate': '2009-01-20', 'end_mandate': '2017-01-20'}, {'id': 'Q91', 'first_name': 'Abraham', 'last_name': 'Lincoln', 'birth_date': '1809-02-12', 'death_date': '1865-04-15', 'place_of_birth': 'Sinking Spring Farm', 'wikipedia_url': 'https://en.wikipedia.org/wiki/Abraham_Lincoln', 'start_mandate': '1861-03-04', 'end_mandate': '1865-04-15'}, {'id': 'Q207', 'first_name': 'George', 'last_name': 'Bush', 'birth_date': '1946-07-06', 'death_date': None, 'place_of_bi

### 1e. A sneak peak at the data (finally)

Look inside a few examples. Notice the dictionary keys available for each entry. You have your first example of *structured data*. This data was sanitized for your benefit, meaning it is readily exploitable without modification.

You will also notice there is a Wikipedia link for each entry. You will need to extract additional information there. This will be a case of *semi-structured* data.

The /countries endpoint returns a `list` of several country codes.

You need to loop through this list and query the /leaders endpoint for each one. Save each `json` result in a dictionary called `leaders_per_country`.

In [43]:
from pprint import pprint
countries_url = f"{root_url}/countries"
countries_response = requests.get(countries_url,cookies = cookies)
countries = countries_response.json() # python listesine cevirdi
# countries = ["us", "fr", "ru", "be", "ma"]


leaders_per_country = {}  
    # {US : [
    # {"first_name": "George", "last_name": "Washington"},
    # {"first_name": "Abraham", "last_name": "Lincoln"},
    # ]
    # }
for country in countries : 
    params = {"country" : country}
    response = requests.get(leaders_url,cookies=cookies,params=params)
    leaders_list = response.json() 
    #leaders_list = [
    # {"first_name": "George", "last_name": "Washington"},
    #{"first_name": "Abraham", "last_name": "Lincoln"},]
    leaders_per_country[country] = leaders_list #dictionary’e kaydediyorsun:
    pprint(leaders_per_country)
    


  


    





{'be': [{'birth_date': '1953-04-11',
         'death_date': None,
         'end_mandate': '2008-03-20',
         'first_name': 'Guy',
         'id': 'Q12978',
         'last_name': 'Verhofstadt',
         'place_of_birth': 'Dendermonde',
         'start_mandate': '1999-07-12',
         'wikipedia_url': 'https://nl.wikipedia.org/wiki/Guy_Verhofstadt'},
        {'birth_date': '1960-10-06',
         'death_date': None,
         'end_mandate': '2011-12-06',
         'first_name': 'Yves',
         'id': 'Q12981',
         'last_name': 'Leterme',
         'place_of_birth': 'Wervik',
         'start_mandate': '2009-11-25',
         'wikipedia_url': 'https://nl.wikipedia.org/wiki/Yves_Leterme'},
        {'birth_date': '1947-10-31',
         'death_date': None,
         'end_mandate': '2009-11-25',
         'first_name': 'Herman',
         'id': 'Q12983',
         'last_name': 'None',
         'place_of_birth': 'Etterbeek',
         'start_mandate': '2008-12-30',
         'wikipedia_url': 'http

It is finally time to create a `get_leaders()` function for the above code. You will build on it later-on. This function takes no parameter. Inside it, you will need to:
1. define the urls
2. get the cookies
2. get the countries
3. loop over them and save their leaders in a dictionary
4. return the dictionary

In [44]:
import requests

def get_leaders():

    root_url = "https://country-leaders.onrender.com"
    cookie_url = f"{root_url}/cookie"
    countries_url = f"{root_url}/countries"
    leaders_url = f"{root_url}/leaders"
    
    # cookie istegi
    response = requests.get(cookie_url) # cookie istegi at
    cookies = response.cookies # gelen cevaptan cookie nesnesini al
    
    #ulkeleri al
    countries_response = requests.get(countries_url,cookies = cookies)
    countries = countries_response.json()
    print(countries)

    # bos sozluk olustur
    leaders_per_country = {}

    #her ulke icin dongu
    for country in countries: 
        params = {"country": country}
        leaders_response = requests.get(leaders_url, cookies = cookies, params = params)  
        if leaders_response.status_code != 200:
            print(f"Error: for {country} no data found ({leaders_response.status_code})")
            continue  # Bu ülkeyi atla
          
        leaders_list = leaders_response.json()
        print(leaders_list) # her ulke icin liderlerin bilgilerini al
        
        #sonucu dict ekle
        leaders_per_country[country] = leaders_list

    return leaders_per_country

get_leaders()










['be', 'us', 'ru', 'fr', 'ma']
[{'id': 'Q12978', 'first_name': 'Guy', 'last_name': 'Verhofstadt', 'birth_date': '1953-04-11', 'death_date': None, 'place_of_birth': 'Dendermonde', 'wikipedia_url': 'https://nl.wikipedia.org/wiki/Guy_Verhofstadt', 'start_mandate': '1999-07-12', 'end_mandate': '2008-03-20'}, {'id': 'Q12981', 'first_name': 'Yves', 'last_name': 'Leterme', 'birth_date': '1960-10-06', 'death_date': None, 'place_of_birth': 'Wervik', 'wikipedia_url': 'https://nl.wikipedia.org/wiki/Yves_Leterme', 'start_mandate': '2009-11-25', 'end_mandate': '2011-12-06'}, {'id': 'Q12983', 'first_name': 'Herman', 'last_name': 'None', 'birth_date': '1947-10-31', 'death_date': None, 'place_of_birth': 'Etterbeek', 'wikipedia_url': 'https://nl.wikipedia.org/wiki/Herman_Van_Rompuy', 'start_mandate': '2008-12-30', 'end_mandate': '2009-11-25'}, {'id': 'Q14989', 'first_name': 'Léon', 'last_name': 'Delacroix', 'birth_date': '1867-12-27', 'death_date': '1929-10-15', 'place_of_birth': 'Saint-Josse-ten-Noode

{'be': [{'id': 'Q12978',
   'first_name': 'Guy',
   'last_name': 'Verhofstadt',
   'birth_date': '1953-04-11',
   'death_date': None,
   'place_of_birth': 'Dendermonde',
   'wikipedia_url': 'https://nl.wikipedia.org/wiki/Guy_Verhofstadt',
   'start_mandate': '1999-07-12',
   'end_mandate': '2008-03-20'},
  {'id': 'Q12981',
   'first_name': 'Yves',
   'last_name': 'Leterme',
   'birth_date': '1960-10-06',
   'death_date': None,
   'place_of_birth': 'Wervik',
   'wikipedia_url': 'https://nl.wikipedia.org/wiki/Yves_Leterme',
   'start_mandate': '2009-11-25',
   'end_mandate': '2011-12-06'},
  {'id': 'Q12983',
   'first_name': 'Herman',
   'last_name': 'None',
   'birth_date': '1947-10-31',
   'death_date': None,
   'place_of_birth': 'Etterbeek',
   'wikipedia_url': 'https://nl.wikipedia.org/wiki/Herman_Van_Rompuy',
   'start_mandate': '2008-12-30',
   'end_mandate': '2009-11-25'},
  {'id': 'Q14989',
   'first_name': 'Léon',
   'last_name': 'Delacroix',
   'birth_date': '1867-12-27',
   'd

Test your function, save the result in the `leaders_per_country` dictionary and check its ouput.

Please set a user-agent and respect our robot policy https://w.wiki/4wJS. See also T400119.



## 2. Extracting data from Wikipedia

Query one of the leaders' Wikipedia urls and display its `text` (not JSON).

In [45]:
import requests

url = "https://nl.wikipedia.org/wiki/Yves_Leterme"

headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64)"}

response = requests.get(url, headers = headers)

print(response.status_code)

print(response.text)  # wikipediada metin / API JSON veriyordu



200
<!DOCTYPE html>
<html class="client-nojs vector-feature-language-in-header-enabled vector-feature-language-in-main-page-header-disabled vector-feature-page-tools-pinned-disabled vector-feature-toc-pinned-clientpref-1 vector-feature-main-menu-pinned-disabled vector-feature-limited-width-clientpref-1 vector-feature-limited-width-content-enabled vector-feature-custom-font-size-clientpref-1 vector-feature-appearance-pinned-clientpref-1 vector-feature-night-mode-enabled skin-theme-clientpref-day vector-sticky-header-enabled vector-toc-available" lang="nl" dir="ltr">
<head>
<meta charset="UTF-8">
<title>Yves Leterme - Wikipedia</title>
<script>(function(){var className="client-js vector-feature-language-in-header-enabled vector-feature-language-in-main-page-header-disabled vector-feature-page-tools-pinned-disabled vector-feature-toc-pinned-clientpref-1 vector-feature-main-menu-pinned-disabled vector-feature-limited-width-clientpref-1 vector-feature-limited-width-content-enabled vector-fe

Ouch! You get the raw HTML code of the webpage. If you try to deal with it without tools, you will be there all night. Instead, use the [beautiful soup 4](https://www.crummy.com/software/BeautifulSoup/bs4/doc/) *external* library. You will find more info about it [here](../../2.python/2.python_advanced/05.Scraping/1.beautifulsoup_basic.ipynb) and [here](../../2.python/2.python_advanced/05.Scraping/2.beautifulsoup_advanced.ipynb)

Using the Quickstart section, start by importing the library and loading the output of your `get_text()` function.

Use the `prettify()` function and print it to take a look. You will start the actual parsing in the next step.

In [ ]:
import requests
from bs4 import BeautifulSoup  

url = "https://nl.wikipedia.org/wiki/Yves_Leterme"

headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64)"}

response = requests.get(url, headers = headers)

content = response.text  

soup = BeautifulSoup(content, "html.parser") # html text treeye cevirdi

p_tag = soup.find("p") # ilk <p> etiketini verir

print(p_tag)  # ilk <p> li etiket
print(p_tag.text.strip()) # <p> li etiketin texti

p_tags = soup.find_all("p")  # tum <p> leri verir

print(p_tags)

soup.find_all("a") # tum <a> taglarini bulur  
for link in soup.find_all("a"):
    print(link.get("href"))   # her <a> tag’indeki href değerini yani linkleri verir
    
for link in soup.find_all("a"):
    href = link.get("href")
    if href:  # eğer None değilse
        print("https://nl.wikipedia.org" + href)
    print(link.text)

    print(link.text) # linkin gorunen adi



That looks better but you need to extract the right part of the webpage: the text of the first paragraph.

It is a bit tricky because Wikipedia pages slightly differ in structure from one language to the next. We cannot simply get the text for the first HTML paragraph.

You will start by getting all the HTML paragraphs from the HTML source and saving them in the `paragraphs` variable.

Use the documentation or google the appropriate keywords.

In [ ]:
import requests
from bs4 import BeautifulSoup  # verilen bir url den paragaraf alma

# 1 Wikipedia sayfasının URL’si
url = "https://nl.wikipedia.org/wiki/Yves_Leterme"

# 2 Wikipedia 403 hatası vermesin diye headers ekliyoruz
headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64)"}

# 3 Sayfayı çekiyoruz
response = requests.get(url, headers=headers)
print(response.status_code)  # 200 olmalı


# 4 BeautifulSoup ile HTML'i parse ediyoruz
soup = BeautifulSoup(response.text, "html.parser")

paragraphs = soup.find_all("p")

# Tüm <p> taglarını sırayla kontrol et
for p in soup.find_all("p"):
    text = p.get_text(strip=True)
    if len(text) > 20:  # anlamlı bir paragrafın uzunluğu 20+ karakter
        first_paragraph = text
        break

print(first_paragraph)





200
Yves Camille Désiré Leterme(Wervik,6 oktober1960) is een voormaligBelgischpoliticus. Hij was onder meerVlaams minister-presidentenpremier van België. Hij was lang boegbeeld van deCD&Ven boegbeeld van het toenmalige kartelCD&V/N-VA.


If you try different urls, you might find that the paragraph you want may be at a different index each time.

That is where you need to be clever and ask yourself what would be a reliable way to identify the right index ie. which string matches only the first paragraph whatever the language...

Spend a good 30 minutes on the problem and brainstorm with your fellow learners. If you come out empty handed, ask your coach.

1. Loop over the HTML paragraphs
2. When you have identified the correct one:
   * Store the [text](https://www.crummy.com/software/BeautifulSoup/bs4/doc/#output) inside the `first_paragraph` variable
   * Exit the loop

At this stage, you can create a function to maintain consistency in your code. We will give you its *skeleton*, you will copy the code you wrote and make it work inside a function.

Don't forget to test your function.

In [57]:
 
import requests
from bs4 import BeautifulSoup 


def get_first_paragraph(wiki_url):
    
    headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64)"}

    try:
        response = requests.get(wiki_url, headers=headers)
        response.raise_for_status()
    except requests.RequestException as e:
        print(f"Error fetching {wiki_url}: {e}")
        return 

    response = requests.get(wiki_url, headers=headers)

    if response.status_code != 200:
        print(f"Error: {wiki_url} returned {response.status_code}")
        return None

    soup = BeautifulSoup(response.text, "html.parser")

    paragraphs = soup.find_all("p")

    birth_words = ["geboren", "born", "né", "nacido"]  # buna bak

    for p in paragraphs:
        text = p.get_text().strip()
        if len(text) < 20:  # cok kisa parag atla
            continue
    
        for word in birth_words :
            if word in text :
                return text
    return None

url = "https://nl.wikipedia.org/wiki/Yves_Leterme"
first_parag = get_first_paragraph(url)
print(first_parag)




Hij werd geboren in Wervik, op de grens tussen Frankrijk en België als zoon van een Franstalige vader en een Vlaamse moeder. Hij bracht zijn jeugd door in het nabijgelegen dorp Zillebeke, een deelgemeente van Ieper.


In [16]:
# Test: 3 lines


### 2a. Regular expressions to the rescue

Now that you have extracted the content of the first paragraph, the only thing that remains to finish your Wikipedia scraper is to sanitize the output.

Indeed some Wikipedia references, HTML code, phonetic pronunciation etc. may linger. You might find *regular expressions* handy to get rid of them and obtain pristine text. You will find some useful documentation about regular expressions [here](../../2.python/2.python_advanced/03.Regex/regex.ipynb)

Once you have one of your regex working online, try it in the cell below. 

Hints: 
* Check the `sub()` method documentation.
* Make sure to test urls in different languages. Some may look good but other do not.

In [17]:
# 3 lines



Overwrite the `get_first_paragraph()` function by applying your regex to the first paragraph before returning it.

In [18]:
# 10 lines


Come up with other regexes to capture other patterns and sanitize the outputs completely. Modify your `get_first_paragraph()` function accordingly.

In [19]:
# < 20 lines


## 3. Putting it all together

Let's go back to your `get_leaders()` function and update it with an *inner* loop over each leader. You will query the url provided and extract the first paragraph using the `get_first_paragraph()` function you just finished. You will then update that `leader`'s dictionary and move on to the next one.

Notice, the rest of the code should not change since you modify the leader's data one by one.

In [ ]:
import requests
from bs4 import BeautifulSoup
from pprint import pprint

def get_leaders():    # 

    root_url = "https://country-leaders.onrender.com"
    cookie_url = f"{root_url}/cookie"
    countries_url = f"{root_url}/countries"
    leaders_url = f"{root_url}/leaders"
    
    # cookie istegi
    response = requests.get(cookie_url) # cookie istegi at
    cookies = response.cookies # gelen cevaptan cookie nesnesini al
    
    headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64)"}  # <<< EKLENDİ
    
    #ulkeleri al
    countries_response = requests.get(countries_url,cookies = cookies)
    countries = countries_response.json()
    print("Countries:", countries)

    # bos sozluk olustur
    leaders_per_country = {}

    #her ulke icin dongu
    for country in countries: 
        params = {"country": country}
        leaders_response = requests.get(leaders_url, cookies = cookies, params = params)  

        if leaders_response.status_code != 200:
            print(f"Error: for {country} no data found ({leaders_response.status_code})")
            continue  # Bu ülkeyi atla
          
        leaders_list = leaders_response.json()
        print(leaders_list) # her ulke icin liderlerin bilgilerini al  
        # bu leaders_list 
        #[
        #{"first_name": "George", "last_name": "Washington", "wikipedia_url": "..."},
        #"{"first_name": "Abraham", "last_name": "Lincoln", "wikipedia_url": "..."}
        #]
        
        # her lider bir sozluk
        
        for leader in leaders_list:  # loop icin eklenen kisim listede don  
            wiki_url = leader.get("wikipedia_url")
            if wiki_url:
                first_paragraph = get_first_paragraph(wiki_url)
                leader["bio_first_paragraph"] = first_paragraph
            else:
                leader["bio_first_paragraph"] = ""  # Wikipedia yoksa boş string
                
        #sonucu dict ekle
        leaders_per_country[country] = leaders_list

    return leaders_per_country

leaders = get_leaders()
pprint(leaders)










In [ ]:
# en son deneme all together biyogrzfinin ilk paragrafi

 
import requests
from bs4 import BeautifulSoup 
from pprint import pprint


def get_first_paragraph(wiki_url):
    
    headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64)"}

    try:
        response = requests.get(wiki_url, headers=headers)
        response.raise_for_status()
    except requests.RequestException as e:
        print(f"Error fetching {wiki_url}: {e}")
        return None # << EKLENDİ: Hata varsa None dön (önceden boş dönüyordu)

    soup = BeautifulSoup(response.text, "html.parser")
    
    # <<< EKLENDİ: Hedef, biyografi başlığı (tüm dillerde) altındaki ilk paragrafı almak
    bio_headers = ["Biography", "Biographie", "Biografía", "Levensloop", "Биография", "生平", "Histoire", "Vita"]  # örnek diller

    # 1 Önce H2 biyografi başlıkları
    for h2 in soup.find_all("h2"):
        h2_text = h2.get_text().strip()
        for bio_title in bio_headers:
            if bio_title.lower() in h2_text.lower():
                next_p = h2.find_next("p")
                if next_p and len(next_p.get_text().strip()) > 20:
                    return next_p.get_text().strip()
    

    #  If no Biography section, get first paragraph under ANY H2 (skip introduction)
    first_h2 = soup.find("h2")
    if first_h2:
        next_p = first_h2.find_next("p")
        if next_p and len(next_p.get_text().strip()) > 20:
            return next_p.get_text().strip()

    
    return None  # hiç paragraf yoksa None dön


def get_leaders():    # 

    root_url = "https://country-leaders.onrender.com"
    cookie_url = f"{root_url}/cookie"
    countries_url = f"{root_url}/countries"
    leaders_url = f"{root_url}/leaders"
    
    # cookie istegi
    response = requests.get(cookie_url) # cookie istegi at
    cookies = response.cookies # gelen cevaptan cookie nesnesini al
    
    headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64)"}  # <<< EKLENDİ
    
    #ulkeleri al
    countries_response = requests.get(countries_url,cookies = cookies)
    countries = countries_response.json()
    print("Countries:", countries)

    # bos sozluk olustur
    leaders_per_country = {}

    #her ulke icin dongu
    for country in countries: 
        # <<< EKLENDİ: Her ülke için cookie yenile (403 hatasını önler)
        response = requests.get(cookie_url, headers=headers)
        cookies = response.cookies  

        params = {"country": country}
        leaders_response = requests.get(leaders_url, cookies = cookies, params = params)  

        if leaders_response.status_code != 200:
            print(f"Error: for {country} ({leaders_response.status_code})")
            continue  # Bu ülkeyi atla
          
        leaders_list = leaders_response.json()
        print(leaders_list) # her ulke icin liderlerin bilgilerini al  
        # bu leaders_list 
        #[
        #{"first_name": "George", "last_name": "Washington", "wikipedia_url": "..."},
        #"{"first_name": "Abraham", "last_name": "Lincoln", "wikipedia_url": "..."}
        #]
        
        # her lider bir sozluk
        # Her lider için Wikipedia paragrafını ekle
        for leader in leaders_list:  
            wiki_url = leader.get("wikipedia_url")
            if wiki_url:
                first_paragraph = get_first_paragraph(wiki_url)
                # <<< DÜZELTİLDİ: boş ise boş string ata
                leader["bio_first_paragraph"] = first_paragraph if first_paragraph else ""
            else:
                leader["bio_first_paragraph"] = ""
                
        #sonucu dict ekle
        leaders_per_country[country] = leaders_list

    return leaders_per_country

leaders = get_leaders()
pprint(leaders)









Countries: ['be', 'us', 'ru', 'fr', 'ma']
[{'id': 'Q12978', 'first_name': 'Guy', 'last_name': 'Verhofstadt', 'birth_date': '1953-04-11', 'death_date': None, 'place_of_birth': 'Dendermonde', 'wikipedia_url': 'https://nl.wikipedia.org/wiki/Guy_Verhofstadt', 'start_mandate': '1999-07-12', 'end_mandate': '2008-03-20'}, {'id': 'Q12981', 'first_name': 'Yves', 'last_name': 'Leterme', 'birth_date': '1960-10-06', 'death_date': None, 'place_of_birth': 'Wervik', 'wikipedia_url': 'https://nl.wikipedia.org/wiki/Yves_Leterme', 'start_mandate': '2009-11-25', 'end_mandate': '2011-12-06'}, {'id': 'Q12983', 'first_name': 'Herman', 'last_name': 'None', 'birth_date': '1947-10-31', 'death_date': None, 'place_of_birth': 'Etterbeek', 'wikipedia_url': 'https://nl.wikipedia.org/wiki/Herman_Van_Rompuy', 'start_mandate': '2008-12-30', 'end_mandate': '2009-11-25'}, {'id': 'Q14989', 'first_name': 'Léon', 'last_name': 'Delacroix', 'birth_date': '1867-12-27', 'death_date': '1929-10-15', 'place_of_birth': 'Saint-Joss

In [64]:
# basit ilk paragrafli olan

import requests
from bs4 import BeautifulSoup 
from pprint import pprint


def get_first_paragraph(wiki_url):
    
    headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64)"}

    try:
        response = requests.get(wiki_url, headers=headers)
        response.raise_for_status()
    except requests.RequestException as e:
        print(f"Error fetching {wiki_url}: {e}")
        return None # << EKLENDİ: Hata varsa None dön (önceden boş dönüyordu)

    soup = BeautifulSoup(response.text, "html.parser")
    
    # <<< EKLENDİ: Hedef, biyografi başlığı (tüm dillerde) altındaki ilk paragrafı almak
    bio_headers = ["Biography", "Biographie", "Biografía", "Levensloop", "Биография", "生平", "Histoire", "Vita"]  # örnek diller

    # 1 Önce H2 başlıkları arasında biyografi başlığı ara
    for h2 in soup.find_all("h2"):
        h2_text = h2.get_text().strip()
        for bio_title in bio_headers:
            if bio_title.lower() in h2_text.lower():
                next_p = h2.find_next("p")
                if next_p:
                    text = next_p.get_text().strip()
                    if len(text) > 50:  # çok kısa paragraf atla
                        return text

    # 2 Eğer H2 biyografi yoksa, sayfadaki ilk uzun paragrafı al
    for p in soup.find_all("p"):
        text = p.get_text().strip()
        if len(text) > 50:  # çok kısa paragraf atlama
            return text
    return None  # hiç paragraf yoksa None dön


def get_leaders():    

    root_url = "https://country-leaders.onrender.com"
    cookie_url = f"{root_url}/cookie"
    countries_url = f"{root_url}/countries"
    leaders_url = f"{root_url}/leaders"
    
    # cookie istegi
    response = requests.get(cookie_url) # cookie istegi at
    cookies = response.cookies # gelen cevaptan cookie nesnesini al
    
    headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64)"}  # <<< EKLENDİ
    
    #ulkeleri al
    countries_response = requests.get(countries_url,cookies = cookies)
    countries = countries_response.json()
    print("Countries:", countries)

    # bos sozluk olustur
    leaders_per_country = {}

    #her ulke icin dongu
    for country in countries: 
        # <<< EKLENDİ: Her ülke için cookie yenile (403 hatasını önler)
        response = requests.get(cookie_url, headers=headers)
        cookies = response.cookies  

        params = {"country": country}
        leaders_response = requests.get(leaders_url, cookies = cookies, params = params)  

        if leaders_response.status_code != 200:
            print(f"Error: for {country} ({leaders_response.status_code})")
            continue  # Bu ülkeyi atla
          
        leaders_list = leaders_response.json()
        print(leaders_list) # her ulke icin liderlerin bilgilerini al  
        # bu leaders_list 
        #[
        #{"first_name": "George", "last_name": "Washington", "wikipedia_url": "..."},
        #"{"first_name": "Abraham", "last_name": "Lincoln", "wikipedia_url": "..."}
        #]
        
        # her lider bir sozluk
        # Her lider için Wikipedia paragrafını ekle
        for leader in leaders_list:  
            wiki_url = leader.get("wikipedia_url")
            if wiki_url:
                first_paragraph = get_first_paragraph(wiki_url)
                # <<< DÜZELTİLDİ: boş ise boş string ata
                leader["bio_first_paragraph"] = first_paragraph if first_paragraph else ""
            else:
                leader["bio_first_paragraph"] = ""
                
        #sonucu dict ekle
        leaders_per_country[country] = leaders_list

    return leaders_per_country

leaders = get_leaders()
pprint(leaders)











Countries: ['be', 'us', 'ru', 'fr', 'ma']
[{'id': 'Q12978', 'first_name': 'Guy', 'last_name': 'Verhofstadt', 'birth_date': '1953-04-11', 'death_date': None, 'place_of_birth': 'Dendermonde', 'wikipedia_url': 'https://nl.wikipedia.org/wiki/Guy_Verhofstadt', 'start_mandate': '1999-07-12', 'end_mandate': '2008-03-20'}, {'id': 'Q12981', 'first_name': 'Yves', 'last_name': 'Leterme', 'birth_date': '1960-10-06', 'death_date': None, 'place_of_birth': 'Wervik', 'wikipedia_url': 'https://nl.wikipedia.org/wiki/Yves_Leterme', 'start_mandate': '2009-11-25', 'end_mandate': '2011-12-06'}, {'id': 'Q12983', 'first_name': 'Herman', 'last_name': 'None', 'birth_date': '1947-10-31', 'death_date': None, 'place_of_birth': 'Etterbeek', 'wikipedia_url': 'https://nl.wikipedia.org/wiki/Herman_Van_Rompuy', 'start_mandate': '2008-12-30', 'end_mandate': '2009-11-25'}, {'id': 'Q14989', 'first_name': 'Léon', 'last_name': 'Delacroix', 'birth_date': '1867-12-27', 'death_date': '1929-10-15', 'place_of_birth': 'Saint-Joss

Does the function crash in the middle of the loop? Chances are the cookies have expired while looping over the leaders.

Modify your function with an *exception* or check if the `status_code` is a cookie error. In either case, get new cookies and query the api again.

If your code did not crash,

In [22]:
# < 25 lines



Check the output of your function again.

In [23]:
# Check the output of your function (1 line)


Well done! It took a while however... Let's speed things up. The main *bottleneck* is the loop. We call on the Wikipedia website many times.

You will use the same *session* to call all the wikipedia pages. Check the *Advanced Usage* section of the Requests module's documentation.

Start by modifying the `get_first_paragraph()` function to accept a session parameter and adjust the `get()` method call.

In [24]:
# < 20 lines


Modify your `get_leaders()` function to make use of a single session for all the Wikipedia calls.
1. create a `Session` object outside of the loop over countries.
2. pass it to the `get_first_paragraph()` function as an argument.

In [25]:
# <25 lines


Test your new functions.



## 4. Saving your hard work

The final step is to save the ``leaders_per_country`` dictionary in the `leaders.json` file using the [json](https://docs.python.org/3/library/json.html) module. Check out the `with` statement.

In [26]:
# 3 lines


Make sure the file can be read back. Write the code to read the file. And check the variables are the same.

In [27]:
# 3 lines


Make a function `save(leaders_per_country)` to call this code easily.

In [28]:
# 3 lines


In [29]:
# Call the function (1 line)


## 5. Tidy things up in a stand-alone python script

Congratulations! You now have a working scraper! However, your code is scattered throughout this notebook along side the tutorials. Hardly production ready...

Copy and paste what you need in a separate `leaders_scraper.py` file.
Make sure it works by calling `python3 leaders_scraper.py`

## (Optional) To go further

If you want to practice scraping, you can read this section and tackle the exercises.

1. Restructure your code by using OOP (see ReadMe).
2. You have noticed the API returns very partial results for country leaders. Many are missing. Overwrite the `get_leaders()` function to get its list from Wikipedia and extract their *personal details* from the frame on the side.

Good luck!